In [1]:
from datetime import datetime 
import pandas as pd
import sqlalchemy as db
import numpy as np
import pyodbc

In [2]:
#Conection with your data base
conn = pyodbc.connect('Driver={Teradata Database ODBC Driver 16.20};'
                        'DBCNAME={your_db_name};'
                        'UID={user};'
                        'PWD={password}')


consulta = conn.cursor()
consulta.execute('''
    CREATE VOLATILE MULTISET TABLE TEMP_TC_HISTORICO_APROBADA_ALERT AS
        (
        SELECT  
            A.idSeccion AS TIENDA,
            A.registro  AS REGISTRO,
            CONCAT(A.apPaterno,' ',A.apMaterno,' ',A.nombres)  AS NOMBRE_COLABORADOR,
            A.categoria AS CATEGORIA, 
            A.estado AS ESTADO,
            A.tipo AS TIPO 
        FROM DLAB_DESNEGRET.DB_FFLL_COLABORADOR A
        WHERE idseccion = 24100 and  fecMaxRed = '1900-01-01'
        )

    WITH DATA 
    PRIMARY INDEX(EJECUTIVO,NRODOCTIT ) ON COMMIT PRESERVE ROWS;
        ''')
    
consulta.commit()
consulta.close()
    

query_temporal = '''
    SELECT FECRESOL, COUNT(*) CANTIDAD from TEMP_TC_HISTORICO_APROBADA_ALERT
    WHERE MESRESOL=(year(date)*100+month(date))
    AND CODESTADO=3 AND CODOPERAC IN (1,2)
    GROUP BY FECRESOL
    ORDER BY FECRESOL DESC;
    '''

data_avance = pd.read_sql_query(query_temporal,conn)

C:\Users\B41319\AppData\Local\Temp\ipykernel_24420\394147558.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_avance = pd.read_sql_query(query_temporal,conn)


In [3]:
data_avance

,TIENDA,REGISTRO,NOMBRE_COLABORADOR,CATEGORIA,ESTADO,TIPO
0,24100,B41775,TIPISMANA TAPULLIMA MARIA STEFANI,RF DIGITAL,OCUPADA,CUBIERTO
1,24100,B35292,ORE CASTILLO ADRIANA YSABEL,RF EXPERTO,OCUPADA,CUBIERTO
2,24100,B35898,ARCE BRICEÑO SAHORY STEFANIA,GERENTE ASISTENTE TDA IB LIMA,OCUPADA,CUBIERTO
3,24100,B37547,GUARDERAS GARAY ALEXIS RICARDO,RF EXPERTO,OCUPADA,CUBIERTO
4,24100,B26197,CRISOLOGO CCAPA LINDA,RF SENIOR GUIA,OCUPADA,CUBIERTO
